In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pygcransac
from time import time

correspondences = np.loadtxt('img/pose6dscene_points.txt')
gt_pose = np.loadtxt('img/pose6dscene_gt.txt')
intrinsic_params = np.loadtxt('img/pose6dscene.K')

print("Number of correspondences loaded = ", str(len(correspondences)))

Number of correspondences loaded =  95


In [7]:
def verify_pygcransac(corrs, K):    
    n = len(correspondences)
    imagePoints = np.float32([corrs[i][0:2] for i in np.arange(n)]).reshape(-1,2)
    worldPoints = np.float32([corrs[i][2:5] for i in np.arange(n)]).reshape(-1,3)
    
    threshold = 5.5
    normalized_threshold = threshold / (K[0, 0] + K[1, 1]) / 2.0;    
    pose, mask = pygcransac.find6DPose(
        np.ascontiguousarray(imagePoints), # 2D points in the image
        np.ascontiguousarray(worldPoints), # 3D points in the world
        probabilities = [], # Inlier probabilities. This is not used if the sampler is not 3 (NG-RANSAC) or 4 (AR-Sampler)
        sampler = 0, # Sampler index (0 - Uniform, 1 - PROSAC, 2 - P-NAPSAC, 3 - NG-RANSAC, 4 - AR-Sampler)
        threshold = normalized_threshold,  # Inlier-outlier threshold
        conf = 0.999) # RANSAC confidence
    return pose, mask

def normalize_image_points(corrs, K): 
    n = len(corrs)
    normalized_correspondences = np.zeros((corrs.shape[0], 5))
    inv_K = np.linalg.inv(K)

    for i in range(n):
        p1 = np.append(correspondences[i][0:2], 1)
        p2 = inv_K.dot(p1)
        normalized_correspondences[i][0:2] = p2[0:2]
        normalized_correspondences[i][2:] = correspondences[i][2:]
    return normalized_correspondences

def calculate_error(gt_pose, est_pose):
    
    R2R1 = np.dot(gt_pose[:, 0:3].T, est_pose[:, 0:3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:, 3] - est_pose[:, 3])
    
    return err_R, err_t


In [8]:
normalized_correspondences = normalize_image_points(correspondences, intrinsic_params)

t = time()

mag_pose, mag_mask = verify_pygcransac(normalized_correspondences, intrinsic_params)
print (time()-t, ' sec gc-ransac')

err_R, err_t = calculate_error(gt_pose, mag_pose)

print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' mm')

0.0123291015625  sec gc-ransac
Rotation error =  1.8783020217535148e-05 °
Translation error =  0.0001007179771463513  mm
